[View in Colaboratory](https://colab.research.google.com/github/derekwise/math/blob/master/PermutationGroups_Numpy.ipynb)

# Fast Permutation Group Computations in Numpy

**Derek K. Wise**

One common notation for a permutation of $n$ elements is simply given by indexing the original elements in order, and then listing these indices in the order they appear after permutation.  For example, this list:
$$[1,3,0,2]$$ 
denotes the permutation given by
$$\begin{array}{ccc}
 0 &\mapsto& 1 \\
 1 &\mapsto& 3 \\
 2 &\mapsto& 0 \\
 3 &\mapsto& 2 
\end{array}
$$

In Python, we could represent such a permutation by simply using the `list` data structure.  However, it is really convenient to to use numpy arrays instead, for these reasons: 

1. The group's multiplication is simply composition using numpy's "advanced indexing" features.
2. The inverse operation is simply numpy's built-in `argsort` method.
3. If you need to do *many* group operations simultaneously, the [vectorization](https://en.wikipedia.org/wiki/Array_programming) of numpy arrays makes such calculations very efficient.  

In [0]:
import numpy as np

## Permutation arrays

Let's define a **permuations array** to be a 1d numpy array whose entries are some permutation of the numbers $0, 1, ... , (n-1)$ for some natural number $n$.  So, a typical permuation array looks like this:

In [2]:
np.array([1,3,0,2])

array([1, 3, 0, 2])

## Multiplication is "advanced indexing"

The simplest kind of [advanced indexing](https://docs.scipy.org/doc/numpy-1.13.0/reference/arrays.indexing.html#advanced-indexing) in numpy is something like `x[[2,0,1]]`, where `x` is a 1d array.  This returns a new 1d array whose elements are elements of `x` in the order 2, 0, 1.  For example, if we let `x = array(['a','b','c'])`,  then `x[[2,0,1]] = array(['c','a','b'])`.  This also works when the index array, `[2,0,1]` in this case, is a numpy array rather than a list:



In [3]:
x = np.array(['a','b','c'])
p = np.array([2,0,1])

print(x[p])

['c' 'a' 'b']


In other words, at least in the case where `x` is a 1d array and `p` is a permutation array of the same length `x[p]` is just the permutation `p` applied to the entries of `x`.

This lets us use permutation arrays to permutate the entries of other arrays (see <a href="#actions">Actions</a>, below).  More importantly a permutation array can permute the entries of another *permutation* array.  This is exactly how multiplication (or composition) of permutation arrays works.  An example: 

In [0]:
x1 = np.array([0,4,3,1,2])
x2 = np.array([4,1,2,0,3])

In [5]:
x1[x2]

array([2, 4, 3, 0, 1])

You can check that this is indeed the result of doing `x2` first and then `x1`. 

### Identity

The identity permutation on $n$ elements is `[0, 1, ... , n-1]`, which in numpy language is just `arange(n)`:

In [6]:
def id(n):
  return np.arange(n)

print(id(4))

[0 1 2 3]


Let's check this on an example. I'll define a permutation array `x` of length 5 and then check that
```
x[id(5)] == x
id(5)[x] == x
```

Since the `==` operator on numpy arrays works componentwise, returning an array of boolean values, it is convenient to enclose the result in an `all( )`, so that we get a single boolean:

In [7]:
x = np.array([0,4,3,1,2])

print(all(x[id(5)] == x))
print(all(id(5)[x] == x))

True
True


### Associativity

Since our group multiplication is $(x,y)\mapsto x[y]$, associativity takes the form $$x[y][z] = x[y[z]]$$
This holds for any permutation arrays of the same size, but let's just check it in Python on one example. 

In [8]:
x1 = np.array([0,4,3,1,2])
x2 = np.array([4,1,2,0,3])
x3 = np.array([2,1,4,3,0])

all(x1[x2[x3]] == x1[x2][x3])

True

### Opposite multiplication convention

For multiplication of permutations, there are two conventions.  The one implicitly picked out is the convention related to the usual *composition* order of functions: namely, that $x \circ y$ means do $y$ first and then do $x$.  This is easy to verify from the example above. 

If we wanted the opposite convention---that multiplying $x\cdot y$ means to first do the permutation $x$ and then do the permutation $gy$---then we could define a custom **opposite multiplication** operation `mult_op`. 

In [0]:
def mult_op(x, y):
  return y[x]

In [10]:
mult_op(x1, x2)

array([4, 3, 0, 1, 2])

Of course, the opposite multiplication is also associative.

In [11]:
all(mult_op(mult_op(x1,x2),x3) == mult_op(x1, mult_op(x2, x3)))

True

## Inversion is `argsort()`

Since the permutation $[a_0,a_1,\ldots a_{n-1}]$ maps $i\mapsto a_i$, the inverse of this permutation, denoted $[a_0,a_1,\ldots a_{n-1}]^{-1}$, maps $a_i \to i$.  

For example, $$[1,3,0,2]^{-1}$$ maps $1 \mapsto 0$, $3 \mapsto 1$, $0\mapsto 2$, and $2\mapsto 3$.   Writing this in the list notation, we find:   
$$[1,3,0,2]^{-1} = [2,0,3,1].$$
If you examine this process carefully, you'll see that inverting a permutation array amounds to listing the *indices* of the elements $0, 1, \ldots$ in order.  This is exactly what numpy's `argsort()` method does! 

The `argsort()` method is of course more general: when applied to a 1d array of numbers, it returns the order of the *indices* needed to sort the array, even if there are  elements that are equal.  But, in our case, the elements *are* indices, and argsort is just the same as inversion.    

In [12]:
np.array([1,3,0,2]).argsort()

array([2, 0, 3, 1])

Let's check that this really is the inverse in this example:

In [13]:
x1 = np.array([1, 3, 0, 2])

print(mult_op(x1, x1.argsort()))
print(mult_op(x1.argsort(), x1))

[0 1 2 3]
[0 1 2 3]


## Generating the permutation group $S_n$

To build the entire permutation group $S_n$, it is convenient to use the `itertools` module, which has a generator for permutations of a list, and returns all of the permutations in lexecographic order with respect to the original order of the list.

In [0]:
from itertools import permutations

def perm_elts(n):
  id = list(range(n))
  perms = list(permutations(id))
  return np.array(list(map(list, perms)))

In [0]:
s3 = perm_elts(3)

In [16]:
print(s3)

[[0 1 2]
 [0 2 1]
 [1 0 2]
 [1 2 0]
 [2 0 1]
 [2 1 0]]


In [17]:
print(s3.argsort())

[[0 1 2]
 [0 2 1]
 [1 0 2]
 [2 0 1]
 [1 2 0]
 [2 1 0]]


## Example: Dihedral group

Of course, *every* finite group is a subgroup of a permutation group, so we can really do caluclations in any finite group in this way. 

The dihedral group $D_n$ is the symmetry group on a regular $n$-gon, and can be seen as a subgroup of the group of permutations of the vertices $[0,\ldots,n-1]$, labeled in, say, clockwise order.  It is generated by two elements: a reflection through vertex "0" and a reflection throught the midpoint of the edge between vertices 0 and 1. These are given respectively by:

$a = [0, n-1, n-2, \ldots 1]$

$b = [1, 0, n-1, n-2, \ldots 1]$

And, since these are both order 2, we can get all $2n$ elements of the group by alternating products of $a$ and $b$.  That's what the generator `dihedral_elts_gen` below does.  The function `dihedral_elts` calls this generator and returns a numpy array populated with all of those group elements. 

In [0]:
def dihedral_elts_gen(n):
  a = np.append(np.array([0]), np.arange(n-1,0,-1) )
  b = np.append(np.array([1,0]), np.arange(n-1,1,-1) )
  w = np.arange(n) # initially the identity permutation
  for _ in range(n):
    yield w
    w = a[w]
    yield w
    w = b[w]
    
def dihedral_elts(n):
  return np.array(list(dihedral_elts_gen(n)))
  
    

In [24]:
print(dihedral_elts(4))

[[0 1 2 3]
 [0 3 2 1]
 [1 2 3 0]
 [3 2 1 0]
 [2 3 0 1]
 [2 1 0 3]
 [3 0 1 2]
 [1 0 3 2]]


## Vectorization

"Vectorized operations" act on an entire array at once, rather than doing the same kind of operation on individual elements one at a time.  This makes them much more efficient than doing things, for example, with for loops.  

We've seen we can do multiplication like this:

In [29]:
x1 = np.array([1,3,2,0])
y1 = np.array([0,2,1,3])
x2 = np.array([2,1,0,3])
y2 = np.array([2,1,3,0])

print(x1[y1])
print(x2[y2])

[1 2 3 0]
[0 1 3 2]


But we can also do this:  First define arrays containing both of the `x` elements and both of the `y` elements, and then use a bit of advanced indexing to multiply.

In [30]:
xs = np.array([x1,x2])
ys = np.array([y1,y2])
print(xs)
print(ys)

[[1 3 2 0]
 [2 1 0 3]]
[[0 2 1 3]
 [2 1 3 0]]


In [31]:
xs[np.array([[0],[1]]), ys]

array([[1, 2, 3, 0],
       [0, 1, 3, 2]])

This is the same as we got before.  

Using this idea, we can define a multiplication operation that operates componentwise on two arrays of permutation arrays.

In [0]:
def vmult(arr1, arr2):
  assert len(arr1.shape) == 2
  assert arr1.shape == arr2.shape
  length = arr1.shape[0]
  row_selector = np.expand_dims(np.arange(length), -1)
  return arr1[row_selector, arr2]

def vmult_op(arr1, arr2):
  return vmult(arr2, arr1)

In [33]:
vmult(xs, ys)

array([[1, 2, 3, 0],
       [0, 1, 3, 2]])


Just as a speed test, let's make an array that containts all $10! = 3{,}628{,}800$ elements $S_{10}$, invert one copy using `argsort()`, and then multiply the two copies together, just to verify that we get $10!$ copies of the identity element. Not a terribly interesting calculation, but a big one, and we're mainly interested in checking the speed. I'll use the `%%time` cell magic to print out the time required for the next two cells.  You can see that it takes much longer just to create the array with all of the group elements than it does to do the actual calculation.  All $10!$ inversions and multiplications take only around a second.   

In [36]:
%%time 
s10 = perm_elts(10)

CPU times: user 9.46 s, sys: 1.32 s, total: 10.8 s
Wall time: 10.7 s


In [37]:
%%time
s10inv = s10.argsort()
prod = vmult(s10,s10inv)
print(prod[:5])

[[0 1 2 3 4 5 6 7 8 9]
 [0 1 2 3 4 5 6 7 8 9]
 [0 1 2 3 4 5 6 7 8 9]
 [0 1 2 3 4 5 6 7 8 9]
 [0 1 2 3 4 5 6 7 8 9]]
CPU times: user 744 ms, sys: 23.5 ms, total: 767 ms
Wall time: 774 ms


** Note: ** Applying multiple permutations to the a single permutation is simpler than the `vmult` function defined above.

In [38]:
print("xs[[0,y1]] = ", xs[[0,y1]], "\n")
print("xs[:,y1] = \n", xs[:,y1])

xs[[0,y1]] =  [1 2 3 0] 

xs[:,y1] = 
 [[1 2 3 0]
 [2 0 1 3]]


## <a name="actions">Actions</a>

Advanced indexing also lets us use a permutation array to act on a given set of symbols.  

For a musical example, there's the action of $D_{12}$ on the set of pitches modulo octave shifts:

In [0]:
notenames = np.array(['C','C#','D','D#','E','F','F#','G','G#','A','A#','B'])

In [0]:
d12 = dihedral_elts(12)

In [42]:
for p in d12:
  print(notenames[p])

['C' 'C#' 'D' 'D#' 'E' 'F' 'F#' 'G' 'G#' 'A' 'A#' 'B']
['C' 'B' 'A#' 'A' 'G#' 'G' 'F#' 'F' 'E' 'D#' 'D' 'C#']
['C#' 'D' 'D#' 'E' 'F' 'F#' 'G' 'G#' 'A' 'A#' 'B' 'C']
['B' 'A#' 'A' 'G#' 'G' 'F#' 'F' 'E' 'D#' 'D' 'C#' 'C']
['D' 'D#' 'E' 'F' 'F#' 'G' 'G#' 'A' 'A#' 'B' 'C' 'C#']
['A#' 'A' 'G#' 'G' 'F#' 'F' 'E' 'D#' 'D' 'C#' 'C' 'B']
['D#' 'E' 'F' 'F#' 'G' 'G#' 'A' 'A#' 'B' 'C' 'C#' 'D']
['A' 'G#' 'G' 'F#' 'F' 'E' 'D#' 'D' 'C#' 'C' 'B' 'A#']
['E' 'F' 'F#' 'G' 'G#' 'A' 'A#' 'B' 'C' 'C#' 'D' 'D#']
['G#' 'G' 'F#' 'F' 'E' 'D#' 'D' 'C#' 'C' 'B' 'A#' 'A']
['F' 'F#' 'G' 'G#' 'A' 'A#' 'B' 'C' 'C#' 'D' 'D#' 'E']
['G' 'F#' 'F' 'E' 'D#' 'D' 'C#' 'C' 'B' 'A#' 'A' 'G#']
['F#' 'G' 'G#' 'A' 'A#' 'B' 'C' 'C#' 'D' 'D#' 'E' 'F']
['F#' 'F' 'E' 'D#' 'D' 'C#' 'C' 'B' 'A#' 'A' 'G#' 'G']
['G' 'G#' 'A' 'A#' 'B' 'C' 'C#' 'D' 'D#' 'E' 'F' 'F#']
['F' 'E' 'D#' 'D' 'C#' 'C' 'B' 'A#' 'A' 'G#' 'G' 'F#']
['G#' 'A' 'A#' 'B' 'C' 'C#' 'D' 'D#' 'E' 'F' 'F#' 'G']
['E' 'D#' 'D' 'C#' 'C' 'B' 'A#' 'A' 'G#' 'G' 'F#' 'F']
['A' 'A#' 

### Left actions

Because `x[y]` in our advanced indexing notation means to apply the permutation `y` to the permutation `x`, we most naturally get a *right* action.  If we want a *left* action instead, we have to use one of two standard tricks.  

In general, to turn a right action $x \lhd y$ into a left action, we define $y \rhd x = x \lhd y^{-1}$.  The inverse is  needed so that the axiom $(x'x) \rhd y = x' \rhd (x \rhd y)$ holds.  

In [0]:
def act(p):
  def f(x):
    return x[p.argsort()]
  return f

In [0]:
p1 = np.array([1,3,2,0])
p2 = np.array([1,0,3,2])
s = np.array(['a','b','c','d'])

In [45]:
# check the main "action" axiom in this example:
print(act(p1)(act(p2)(s)))
print(act(p1[p2])(s))

['c' 'b' 'd' 'a']
['c' 'b' 'd' 'a']


The other alternative is to use not use the inverse, and instead make the action compatible with the opposite multiplication. 

In [0]:
def act_op(p):
  def f(x):
    return x[p]
  return f

In [36]:
print(act_op(p1)(act(p2)(s)))
print(act_op(mult_op(p1,p2))(s))

['a' 'c' 'd' 'b']
['a' 'c' 'd' 'b']
